In [14]:
from sagemaker.huggingface.processing import HuggingFaceProcessor
from sagemaker import get_execution_role

In [15]:
role = get_execution_role()

In [16]:
hf_processor = HuggingFaceProcessor(
    role=role,
    instance_type="ml.p3.2xlarge",
    transformers_version='4.6',
    pytorch_version='1.7',
    instance_count=1,
    base_job_name='sm-hf-model-eval',
)

In [17]:
model = 's3://sm-hf-model-evaluation/model/model.tar.gz'
test_data = 's3://sm-hf-model-evaluation/test_data'

In [18]:
from sagemaker.processing import ProcessingInput, ProcessingOutput
inputs = [ProcessingInput(source=model, destination="/opt/ml/processing/model"),
          ProcessingInput(source=test_data, destination="/opt/ml/processing/test"),]
outputs = [ProcessingOutput(output_name="evaluation", source="/opt/ml/processing/evaluation", destination='s3://sm-hf-model-evaluation/output')]

In [19]:
hf_processor.run(
    code="scripts/evaluate-sm-hf.py",
    inputs=inputs,
    outputs=outputs,
    wait=False,
)


Job Name:  sm-hf-model-eval-2021-08-24-07-52-10-991
Inputs:  [{'InputName': 'input-1', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sm-hf-model-evaluation/model/model.tar.gz', 'LocalPath': '/opt/ml/processing/model', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'input-2', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sm-hf-model-evaluation/test_data', 'LocalPath': '/opt/ml/processing/test', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'code', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-eu-west-2-146178657422/sm-hf-model-eval-2021-08-24-07-52-10-991/source/sourcedir.tar.gz', 'LocalPath': '/opt/ml/processing/input/code/', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'entrypoint', 'AppManaged

In [ ]:
preprocessing_job_description = hf_processor.jobs[-1].describe()

output_config = preprocessing_job_description['ProcessingOutputConfig']
for output in output_config['Outputs']:
    path = output['S3Output']['S3Uri']